In [1]:
import numpy as np
import cv2

def center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy


cv2.startWindowThread()
cap = cv2.VideoCapture('/Users/GAMZE/Desktop/video/test.mp4')
out = cv2.VideoWriter(
    'studycase.avi',
    cv2.VideoWriter_fourcc(*'MJPG'),
    15.,
    (1500,1000))

fgbg = cv2.createBackgroundSubtractorMOG2()

detects = []

cizgi = 150
offset = 30

xy1 = (20, cizgi)
xy2 = (300, cizgi)

toplam = 0
up = 0
down = 0

while (True):
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1500,1000))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    fgmask = fgbg.apply(gray)

    retval, th = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel, iterations = 2)

    dilation = cv2.dilate(opening,kernel,iterations = 8)

    closing = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel, iterations = 8)

    cv2.line(frame, (0, 200), (1500,350), (0, 255, 255), 2)

    contours, hierarchy = cv2.findContours(dilation,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    i = 0
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)

        area = cv2.contourArea(contour)
        
        if int(area) > 3000 :
            cntr = center(x, y, w, h)

            
            
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
            if len(detects) <= i:
                detects.append([])
            if cntr[1]> cizgi-offset and cntr[1] < cizgi+offset:
                detects[i].append(cntr)
            else:
                detects[i].clear()
            i += 1

    if i == 0:
        detects.clear()

    i = 0

    if len(contours) == 0:
        detects.clear()

    else:

        for detect in detects:
            for (c,l) in enumerate(detect):


                if detect[c-1][1] < cizgi and l[1] > cizgi :
                    detect.clear()
                    up+=1
                    toplam+=1
                    cv2.line(frame, (0, 200), (1500,350), (0, 255, 255), 2)
                    continue

                if detect[c-1][1] > cizgi and l[1] < cizgi:
                    detect.clear()
                    down+=1
                    toplam+=1
                    cv2.line(frame, (0, 200), (1500,350), (0, 255, 255), 2)
                    continue

                if c > 0:
                    cv2.line(frame, (0, 200), (1500,350), (0, 255, 255), 2)
                    

    
    cv2.putText(frame, "UP: "+str(up), (200, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255),2)
    cv2.putText(frame, "DOWN: "+str(down), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255),2)
    cv2.putText(frame, "TOPLAM: "+str(toplam), (400, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255),2)
    out.write(frame.astype('uint8'))
    cv2.imshow("studycase", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-q3d_8t8e\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
